In [ ]:
import os
import re
import ast
import numpy as np
import pandas as pd
from pprint import pprint
from tqdm.notebook import tqdm

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel

import torch.nn as nn
from torch.optim import AdamW

In [ ]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1


In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 8
BERT_SPECIES = 'roberta-base'
INPUT_CATS = ['src','rel','trg']
OUTPUT_CATS = ['mis']
EPOCHS = 5
LEARNING_RATE = 1e-5
LOGGING_INTERVAL = 100

In [ ]:
!ls ../input/mismatch-data/syntax-complete_test.csv

In [ ]:
rawdf = {}
for split in ['train','val','test']:
    dpath = f"../../data/complete_{split}.csv"
    rawdf[split] = pd.read_csv(dpath)

In [ ]:
rawdf['train']

In [ ]:
input_cats = INPUT_CATS
output_cats = OUTPUT_CATS
include_context = True

In [ ]:
cat2col = {'src':'SSpan','rel':'RSpan','trg':'TSpan','mis':'MSpan'}
col2cat = {col:cat for cat,col in cat2col.items()}

proc_data = {}
for split in ['train','val','test']:
    proc_data[split] = []
    print(split)
    for _, example in tqdm(rawdf[split].iterrows(),total=len(rawdf[split])):
        set_complete = True
        example_feats = {
            'text': None,
            'input_cols': {},
            'output_cols': {},
        }
#         text = example['data'].strip() if include_context else example['data'].strip().split('\n')[0]
        text = example['claim'] + ' [SEP] '
#         text = example['claim'] + ' <SEP> '

        #         print(text)
        for io_type, cats in {
            'input_cols': input_cats,
            'output_cols': output_cats,
        }.items():
            for cat in cats:
                col = cat2col[cat]
                span = ast.literal_eval(example[col])
                if any([v==None for v in span]):
                    set_complete = False
                else:
                    if cat == 'mis': 
                        span[0] += len(text)
                        span[1] += len(text)
                    example_feats[io_type][col] = span
        text += example['context']
        example_feats['text'] = text
        if set_complete:
            proc_data[split].append(example_feats)

In [ ]:
pprint(proc_data['train'][0])

In [ ]:
proc_data['train'][0]['text'][10:83]


## Method - 1: String Control

In [ ]:
cat_tag_map = {
    'SSpan': ['<SRC>','</SRC>'],
    'RSpan': ['<REL>','</REL>'],
    'TSpan': ['<TRG>','</TRG>'],
    'MSpan': ['<MIS>','</MIS>'],
}

ext_data = {}
for split in ['train','val','test']:
    ext_data[split] = []
    print(split)
    for example in tqdm(proc_data[split]):
        new_text = ""
        new_text_split = []
        new_col_spans = {}          # character level
        new_col_spans_split = {}    # word level
        ptr = 0
        col_span_list = []
        for io_type in ['input_cols','output_cols']:
            for cat, span in example[io_type].items():
                col_span_list.append([io_type,cat,span])
        for io_type, cat, span in sorted(col_span_list,key=lambda x: x[2]):
            new_text += example['text'][ptr:span[0]]
            new_text_split.extend([v for v in example['text'][ptr:span[0]].strip().split(' ') if v!=''])
            ptr = span[0]
            
            cat_start, cat_start_split = len(new_text), len(new_text_split)
            
            if io_type == 'input_cols':
                tags = cat_tag_map[cat]
                string = tags[0] + ' ' + example['text'][span[0]:span[1]] + ' ' + tags[1]
                new_text += string
                new_text_split.extend(string.strip().split())
            else:
                string = example['text'][span[0]:span[1]]
                new_text += string
                new_text_split.extend([v for v in string.strip().split() if v!=''])
                
            cat_end, cat_end_split = len(new_text), len(new_text_split)
            
            new_col_spans[cat] = [cat_start,cat_end]
            new_col_spans_split[cat] = [cat_start_split, cat_end_split]
            
            ptr = span[1]

        new_text += example['text'][ptr:]
        new_text_split.extend(example['text'][ptr:].strip().split())
        
        wid2col = ['TXT' for _ in range(len(new_text_split))]
        for col, w_span in new_col_spans_split.items():
            for wid in range(*w_span):
                wid2col[wid] = col
        
        ext_data[split].append({
            'text': new_text,
            'text_split': new_text_split,
            'input_cols': {col:new_col_spans[col] for col in example['input_cols'].keys()},
            'output_cols': {col:new_col_spans[col] for col in example['output_cols'].keys()},
            'input_cols_split': {col:new_col_spans_split[col] for col in example['input_cols'].keys()},
            'output_cols_split': {col:new_col_spans_split[col] for col in example['output_cols'].keys()},
            'wid2col': wid2col,
        })
#         print(example['text'])
#         print()
#         print(new_text)
#         print()
#         for col, span in new_output_cols.items():
#             print(col)
#             print(new_text[span[0]:span[1]])
        
#         break
#     break

In [ ]:
ext_data['test'][136]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(BERT_SPECIES, add_prefix_space=True)
bert_model = AutoModel.from_pretrained(BERT_SPECIES)

In [ ]:
print(tokenizer.all_special_tokens)
custom_spl_tokens = [tag for cat, tags in cat_tag_map.items() for tag in tags]
# custom_spl_tokens = ['<SEP>'] + [tag for cat, tags in cat_tag_map.items() for tag in tags]
print(custom_spl_tokens)
num_added_toks = tokenizer.add_tokens(custom_spl_tokens, special_tokens=True)
print(num_added_toks)
bert_model.resize_token_embeddings(len(tokenizer))
print(tokenizer.all_special_tokens)
tokenizer.save_pretrained("custom_tokenizer")

In [ ]:
!cat here/added_tokens.json

In [ ]:
enc = {}
for split in ['train','val','test']:
    enc[split] = tokenizer.batch_encode_plus(
    [eg['text_split'] for eg in ext_data[split]],
    max_length=128,
    add_special_tokens=True,
    is_split_into_words=True,
    padding=True,
    truncation=True,
    return_tensors='pt',
    )

In [ ]:
enc['val']['input_ids'].shape

In [ ]:
pipe_data = {} # pipeline-ready data
for split in ['train','val','test']:
    pipe_data[split] = []
    print(split)
    for i in tqdm(range(enc[split]['input_ids'].shape[0])):
        input_ids = enc[split]['input_ids'][i]
        attention_mask = enc[split]['attention_mask'][i]
        sample = ext_data[split][i]
        output_cats = list(sample['output_cols_split'].keys())
        labels = {cat:[] for cat in output_cats}
        wid2col = sample['wid2col']
        for tid, wid in enumerate(enc[split].word_ids(batch_index=i)):
            if wid == None:
                for cat in output_cats:
                    labels[cat].append(0.0)
            else:
                for cat in output_cats:
                    if wid2col[wid] == cat:
                        labels[cat].append(1.0)
                    else:
                        labels[cat].append(0.0)
        pipe_data[split].append({
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels,
        })

In [ ]:
class StringControlDataset(Dataset):
    
    def __init__(self,data):
        self.data = data
        self.output_cats = list(sorted(self.data[0]['labels'].keys()))
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,idx):
        sample = self.data[idx]
        input_ids = sample['input_ids']
        attention_mask = sample['attention_mask']
        labels = torch.tensor([sample['labels'][col] for col in self.output_cats])
        return input_ids, attention_mask, labels

In [ ]:
strctrl_dataset = {}
for split in ['train','val','test']:
    strctrl_dataset[split] = StringControlDataset(pipe_data[split])

In [ ]:
strctrl_dataloader = {}
for split in ['train','val','test']:
    strctrl_dataloader[split] = DataLoader(strctrl_dataset[split],
                                           batch_size=BATCH_SIZE,
#                                            shuffle=True
                                          )

In [ ]:
class StringControlModel(nn.Module):
    def __init__(self,bert_model,out_classes=1):
        super().__init__()
        self.bert_model = bert_model
        self.out_classes = out_classes
        
#         for name, param in self.bert_model.named_parameters():
#              if any([name.startswith(f"encoder.layer.{layer_id}") for layer_id in [0,1,2,
#                                                                     3,4,5,6,7,8,9,] ]):
#                 param.requires_grad = True
        
        self.fc = nn.Sequential(
#                             nn.Dropout(),
                            nn.Linear(768,768),
                            nn.ReLU(),
                        )
        
        self.classifier = nn.Sequential(
#                             nn.Dropout(),
                            nn.Linear(768,self.out_classes),
                            nn.Softmax(dim=1),
                        )

    def forward(self,input_ids,attention_mask):
        emb = self.bert_model(input_ids=input_ids,attention_mask=attention_mask).last_hidden_state
        x = self.fc(emb)
        out = self.classifier(x)
        return out

In [ ]:
model = StringControlModel(bert_model,out_classes=len(strctrl_dataset['train'].output_cats))
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f"trainable params: {trainable_params}")
print(f"    total params: {total_params}")
print(f"                : {round(100*trainable_params/total_params,2)}%")

model = model.to(DEVICE)

In [ ]:
epochs = EPOCHS
loss_fnc = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(),lr=LEARNING_RATE)
logging_interval = LOGGING_INTERVAL

In [ ]:
# Training
for e in range(epochs):
    print(f'running epoch {e+1} of {epochs}')
    model.train()
    batch_loss = []
    epoch_loss = []
    for bidx, batch in tqdm(enumerate(strctrl_dataloader['train']),total=len(strctrl_dataloader['train'])):
        
        input_ids, attention_mask, labels = [v.to(DEVICE) for v in batch]
        out = model(input_ids, attention_mask)
        labels = torch.permute(labels,(0,2,1))
        
        optimizer.zero_grad()
        loss = loss_fnc(out,labels)
        loss.backward()
        optimizer.step()
        
        epoch_loss.append(loss.item())
        batch_loss.append(loss.item())
        if len(batch_loss) > logging_interval:
            batch_loss.pop(0)
        if (bidx+1) % logging_interval == 0:
            print(f'train loss: {np.mean(batch_loss)}')
    print(f"epoch loss: {np.mean(epoch_loss)}")
                    
#         break
#     break

In [ ]:
# Inference
with torch.no_grad():
    model.eval()
    epoch_loss = []
    auto = []
    gold = []
    for bidx, batch in tqdm(enumerate(strctrl_dataloader['train']),
                            total=len(strctrl_dataloader['train'])):

        input_ids, attention_mask, labels = [v.to(DEVICE) for v in batch]
        out = model(input_ids, attention_mask)
        auto.append(out)
        labels = torch.permute(labels,(0,2,1))
        gold.append(labels)
        loss = loss_fnc(out,labels)
        epoch_loss.append(loss.item())
    print(f"epoch loss: {np.mean(epoch_loss)}")

In [ ]:
auto[136].shape, gold[136].shape

In [ ]:
# auto_new = [auto[i][:,:,0].reshape((8,128,1)) for i in range(len(auto))]

In [ ]:
best = -1
best_thresh = None
for q in range(auto[0].shape[2]):
    auto_new = [auto[i][:,:,q].reshape((-1,128,1)) for i in range(len(auto))]
    gold_new = [gold[i][:,:,q].reshape((-1,128,1)) for i in range(len(auto))]
    for thresh in tqdm(np.linspace(0,0.5,200)):
        IOU = []
        for ba, bg in list(zip(auto_new,gold_new))[:100]:
            for i in range(bg.shape[0]):
                sa, sg = ba[i].squeeze(-1) > thresh, bg[i].squeeze(-1) == 1.0
                IOU.append(sum(sa & sg)/(1e-5 + sum(sa | sg)))
        score = torch.mean(torch.tensor(IOU))
        if score > best:
            best = score
            best_thresh = thresh
    print(best, best_thresh)

In [ ]:
# Inference
with torch.no_grad():
    model.eval()
    epoch_loss = []
    auto = []
    gold = []
    for bidx, batch in tqdm(enumerate(strctrl_dataloader['test']),
                            total=len(strctrl_dataloader['test'])):

        input_ids, attention_mask, labels = [v.to(DEVICE) for v in batch]
        out = model(input_ids, attention_mask)
        auto.append(out)
        labels = torch.permute(labels,(0,2,1))
        gold.append(labels)
        loss = loss_fnc(out,labels)
        epoch_loss.append(loss.item())
    print(f"epoch loss: {np.mean(epoch_loss)}")

In [ ]:
tags = ['MSpan']
nbname = 'roberta-b'
for i in range(len(ext_data['test'])):
    ext_data['test'][i]['wid2col_p'] = ext_data['test'][i]['wid2col'][:]
for q in range(auto[0].shape[2]):
    auto_new = [auto[i][:,:,q].reshape((-1,128,1)) for i in range(len(auto))]
    gold_new = [gold[i][:,:,q].reshape((-1,128,1)) for i in range(len(auto))]
    cnt = 0
    for ba, bg in tqdm(zip(auto_new,gold_new)):
        for i in range(bg.shape[0]):
            sa, sg = ba[i].squeeze(-1) > thresh, bg[i].squeeze(-1) == 1.0
            cnt2 = 0
            for a,b in zip(sa, sg):
                if a == True and b == False:
                    ext_data['test'][cnt]['wid2col_p'][cnt2] = tags[q]
                elif a == False and b == True and ext_data['test'][cnt]['wid2col_p'][cnt2] == tags[q]:
                    # print(cnt, cnt2, q)
                    ext_data['test'][cnt]['wid2col_p'][cnt2] = 'TXT'
                elif a == False and b == True:
                    pass
                    # print(cnt, cnt2, q)
                cnt2 += 1
                if cnt2 == len(ext_data['test'][cnt]['wid2col_p']):
                    break
            cnt += 1

In [ ]:
import pickle as pkl
with open('results/' + nbname, 'wb+') as f:
    pkl.dump(ext_data, f)

In [ ]:
for i in range(len(auto)):
    try:
        auto[i][:,:,0].reshape((8,128,1)).shape
    except:
        print(i)

In [ ]:
thresh = float(best_thresh)
IOU = []
for q in range(auto[0].shape[2]):
    auto_new = [auto[i][:,:,q].reshape((-1,128,1)) for i in range(len(auto))]
    gold_new = [gold[i][:,:,q].reshape((-1,128,1)) for i in range(len(auto))]
    for ba, bg in zip(auto_new,gold_new):
        for i in range(bg.shape[0]):
            sa, sg = ba[i].squeeze(-1) > thresh, bg[i].squeeze(-1) == 1.0
            IOU.append(sum(sa & sg)/(1e-5 + sum(sa | sg)))
    #         break
    #     break
    print(torch.mean(torch.tensor(IOU)))

In [ ]:
# Inference
with torch.no_grad():
    model.eval()
    epoch_loss = []
    auto = []
    gold = []
    for bidx, batch in tqdm(enumerate(strctrl_dataloader['val']),
                            total=len(strctrl_dataloader['val'])):

        input_ids, attention_mask, labels = [v.to(DEVICE) for v in batch]
        out = model(input_ids, attention_mask)
        auto.append(out)
        labels = torch.permute(labels,(0,2,1))
        gold.append(labels)
        loss = loss_fnc(out,labels)
        epoch_loss.append(loss.item())
    print(f"epoch loss: {np.mean(epoch_loss)}")

In [ ]:
thresh = float(best_thresh)
IOU = []
for q in range(auto[0].shape[2]):
    auto_new = [auto[i][:,:,q].reshape((-1,128,1)) for i in range(len(auto))]
    gold_new = [gold[i][:,:,q].reshape((-1,128,1)) for i in range(len(auto))]
    for ba, bg in zip(auto_new,gold_new):
        for i in range(bg.shape[0]):
            sa, sg = ba[i].squeeze(-1) > thresh, bg[i].squeeze(-1) == 1.0
            IOU.append(sum(sa & sg)/(1e-5 + sum(sa | sg)))
    #         break
    #     break
    print(torch.mean(torch.tensor(IOU)))